In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path

INPUT_PATH = Path(r"ECON x499.2_ Project 1 Data - Verified Data.csv")
OUTPUT_DIR = Path(".")
df = pd.read_csv(INPUT_PATH)
value_cols = [c for c in df.columns if c != "Metric"]
long = df.melt(id_vars="Metric", value_vars=value_cols, var_name="CompanyYear", value_name="Value")

def parse_company_year(s: str):
    s = s.strip()
    company = "Alphabet" if s.lower().startswith("alphabet") else ("Microsoft" if s.lower().startswith("microsoft") else s.split(" FY ")[0])
    fy = int(s.split("FY")[1].split("(")[0].strip())
    return company, fy

parsed = long["CompanyYear"].apply(parse_company_year)
long["Company"] = [p[0] for p in parsed]
long["FY"] = [p[1] for p in parsed]

wide = long.pivot_table(index=["Company","FY"], columns="Metric", values="Value", aggfunc="first").reset_index()
for col in wide.columns:
    if col not in ["Company","FY"]:
        wide[col] = pd.to_numeric(wide[col], errors="coerce")

wide["Gross Margin"] = wide["Gross Profit"] / wide["Revenue"]
wide["Net Profit Margin"] = wide["Net Income"] / wide["Revenue"]
wide["Current Ratio"] = wide["Total Current Assets"] / wide["Total Current Liabilities"]
wide["Debt-to-Equity"] = wide["Total Debt"] / wide["Total Equity"]

wide_sorted = wide.sort_values(["FY","Company"])

def grouped_bar(data, metric, title, ylabel, filename, percent=False):
    years = sorted(data["FY"].unique())
    companies = ["Alphabet", "Microsoft"]
    x = np.arange(len(companies))
    width = 0.35 if len(years)==2 else 0.25

    fig, ax = plt.subplots(figsize=(8,4.8))
    for i, yr in enumerate(years):
        vals = []
        for comp in companies:
            v = data.loc[(data["Company"]==comp) & (data["FY"]==yr), metric].values
            vals.append(v[0] if len(v)>0 else np.nan)
        offset = (i - (len(years)-1)/2)*width
        ax.bar(x + offset, vals, width, label=f"FY{yr}")

    ax.set_xticks(x)
    ax.set_xticklabels(companies)
    ax.set_title(title)
    ax.set_ylabel(ylabel)
    ax.legend()
    ax.set_ylim(bottom=0)

    for cont in ax.containers:
        for rect in cont:
            h = rect.get_height()
            if np.isnan(h): 
                continue
            label = f"{h*100:.1f}%" if percent else f"{h:.2f}"
            ax.annotate(label, (rect.get_x()+rect.get_width()/2, h),
                        xytext=(0,3), textcoords="offset points", ha='center', va='bottom', fontsize=9)
    fig.tight_layout()
    fig.savefig(OUTPUT_DIR / filename, dpi=200, bbox_inches="tight")
    plt.close(fig)

grouped_bar(wide_sorted, "Gross Margin", "Profitability: Gross Margin", "Gross Margin", "chart_a_gross_margin.png", percent=True)
grouped_bar(wide_sorted, "Net Profit Margin", "Profitability: Net Profit Margin", "Net Profit Margin", "chart_a_net_profit_margin.png", percent=True)
grouped_bar(wide_sorted, "Current Ratio", "Financial Health: Current Ratio", "Current Ratio (x)", "chart_b_current_ratio.png")
grouped_bar(wide_sorted, "Debt-to-Equity", "Financial Health: Debt-to-Equity Ratio", "Debt-to-Equity (x)", "chart_b_debt_to_equity.png")
needed = {"Net cash provided by operating activities", "Purchases of property and equipment"}
available = set(df["Metric"].astype(str))

if needed.issubset(available):
    cash = long.pivot_table(index=["Company","FY"], columns="Metric", values="Value", aggfunc="first").reset_index()
    cash["CFO"] = pd.to_numeric(cash["Net cash provided by operating activities"], errors="coerce")
    cash["CapEx"] = pd.to_numeric(cash["Purchases of property and equipment"], errors="coerce")  # usually negative
    cash["FCF"] = pd.to_numeric(cash["Free Cash Flow"], errors="coerce")
else:
    cash = wide_sorted[["Company","FY","Free Cash Flow"]].rename(columns={"Free Cash Flow":"FCF"}).copy()
    cash["CFO"] = np.nan
    cash["CapEx"] = np.nan

years = sorted(cash["FY"].unique())
companies = ["Alphabet","Microsoft"]
labels, CFO_vals, CapEx_vals, FCF_vals = [], [], [], []

for yr in years:
    for comp in companies:
        row = cash[(cash["FY"]==yr) & (cash["Company"]==comp)].iloc[0]
        labels.append(f"{comp}\nFY{yr}")
        CFO_vals.append(row["CFO"])
        CapEx_vals.append(row["CapEx"])
        FCF_vals.append(row["FCF"])

x = np.arange(len(labels))
w = 0.25
fig, ax = plt.subplots(figsize=(10,5.5))
ax.bar(x - w, CFO_vals, w, label="Cash from Operations (CFO)")
ax.bar(x, CapEx_vals, w, label="CapEx (Purchases of P&E)")
ax.bar(x + w, FCF_vals, w, label="Free Cash Flow (FCF)")
ax.set_xticks(x)
ax.set_xticklabels(labels)
ax.set_title("Cash Generation: CFO − CapEx = FCF")
ax.set_ylabel("$ (millions)")
ax.legend()
ax.axhline(0, linewidth=1)

def annotate(vals, xpos):
    for xi, v in zip(xpos, vals):
        if np.isnan(v):
            ax.annotate("N/A", (xi, 0), xytext=(0,5), textcoords="offset points", ha="center", va="bottom", fontsize=9)
        else:
            ax.annotate(f"{v:,.0f}", (xi, v), xytext=(0,3 if v>=0 else -12), textcoords="offset points",
                        ha="center", va="bottom" if v>=0 else "top", fontsize=9)

annotate(CFO_vals, x - w)
annotate(CapEx_vals, x)
annotate(FCF_vals, x + w)

if not needed.issubset(available):
    ax.text(0.01, -0.18,
            "Note: Your verified sheet includes Free Cash Flow but does not include CFO and CapEx line items, so CFO/CapEx show as N/A.\n"
            "If you add those two metrics (from the 10-K cash flow statement), this chart will populate automatically.",
            transform=ax.transAxes, fontsize=9, va="top")

fig.tight_layout()
fig.savefig(OUTPUT_DIR / "chart_c_cash_generation.png", dpi=200, bbox_inches="tight")
plt.close(fig)

Saved charts:
 - chart_a_gross_margin.png
 - chart_a_net_profit_margin.png
 - chart_b_current_ratio.png
 - chart_b_debt_to_equity.png
 - chart_c_cash_generation.png
